In [5]:
# 
import json
import cv2
import requests
import uuid
from tencentcloud.common import credential
from tencentcloud.common.profile.client_profile import ClientProfile
from tencentcloud.common.profile.http_profile import HttpProfile
from tencentcloud.common.exception.tencent_cloud_sdk_exception import TencentCloudSDKException
from tencentcloud.asr.v20190614 import asr_client
from tencentcloud.asr.v20190614 import models as asr_models
import numpy as np
import base64
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
with open("../tencent_key.json" ,'r') as key:
    SecretDict=json.loads(key.read())
SecretId=SecretDict["SecretId"]
SecretKey=SecretDict["SecretKey"]

In [6]:
"""
https://console.cloud.tencent.com/api
/explorer?Product=asr&Version=2019-06-14&Action=DescribeTaskStatus&SignVersion=
录音文件识别相关接口
"""

video_path="./video/test.wav"
with open(video_path,"rb") as file:

    video=base64.b64encode(file.read()) 
Data=str(video , encoding='utf-8')

try: 
    cred = credential.Credential(SecretId, SecretKey) 
    httpProfile = HttpProfile()
    httpProfile.endpoint = "asr.tencentcloudapi.com"

    clientProfile = ClientProfile()
    clientProfile.httpProfile = httpProfile
    client = asr_client.AsrClient(cred, "", clientProfile) 

    req = asr_models.CreateRecTaskRequest()
    params = {
        "EngineModelType": "16k_zh",
        "ChannelNum": 1,
        "SpeakerDiarization": 0,
        "SpeakerNumber": 0,
        "ResTextFormat": 0,
        "SourceType": 1,
#         "Url": "",
        "Data": Data,
        "FilterDirty": 0,
        "FilterModal": 0,
        "ConvertNumMode": 0,
        "FilterPunc": 0
    }
    req.from_json_string(json.dumps(params))

    resp = client.CreateRecTask(req).to_json_string() 
    resp=json.loads(resp)
    print(resp) 
    TaskId=resp["Data"]["TaskId"]
except TencentCloudSDKException as err: 
    print(err) 

{'Data': {'TaskId': 1529154133}, 'RequestId': '838e7dd7-2ac3-423a-8aa8-b00ddbf18ef4'}


In [7]:
"""
Action	是	String	公共参数，本接口取值：CreateRecTask。
Version	是	String	公共参数，本接口取值：2019-06-14。
Region	否	String	公共参数，本接口不需要传递此参数。
EngineModelType	是	String	引擎模型类型。
电话场景：
• 8k_en：电话 8k 英语；
• 8k_zh：电话 8k 中文普通话通用；
非电话场景：
• 16k_zh：16k 中文普通话通用；
• 16k_zh_video：16k 音视频领域；
• 16k_en：16k 英语；
• 16k_ca：16k 粤语；
• 16k_ja：16k 日语；
• 16k_zh_edu 中文教育；
• 16k_en_edu 英文教育；
• 16k_zh_medical 医疗；
• 16k_th 泰语；
ChannelNum	是	Integer	识别声道数。1：单声道；2：双声道（仅支持 8k_zh 引擎模）。注意：录音识别会自动将音频转码为填写的识别声道数
ResTextFormat	是	Integer	识别结果返回形式。0： 识别结果文本(含分段时间戳)； 1：词级别粒度的详细识别结果(不含标点，含语速值)；2：词级别粒度的详细识别结果（包含标点、语速值）
SourceType	是	Integer	语音数据来源。0：语音 URL；1：语音数据（post body）。
SpeakerDiarization	否	Integer	是否开启说话人分离，0：不开启，1：开启(仅支持8k_zh，16k_zh，16k_zh_video引擎模型，单声道音频)，默认值为 0。
SpeakerNumber	否	Integer	说话人分离人数（需配合开启说话人分离使用），取值范围：0-10，0代表自动分离（目前仅支持≤6个人），1-10代表指定说话人数分离。默认值为 0。
注：话者分离目前是beta版本，请根据您的需要谨慎使用
CallbackUrl	否	String	回调 URL，用户自行搭建的用于接收识别结果的服务URL。如果用户使用轮询方式获取识别结果，则无需提交该参数。回调格式&内容详见：录音识别回调说明
Url	否	String	语音的URL地址，需要公网可下载。长度小于2048字节，当 SourceType 值为 0 时须填写该字段，为 1 时不需要填写。注意：请确保录音文件时长在5个小时之内，否则可能识别失败。请保证文件的下载速度，否则可能下载失败。
Data	否	String	语音数据，当SourceType 值为1时必须填写，为0可不写。要base64编码(采用python语言时注意读取文件应该为string而不是byte，以byte格式读取后要decode()。编码后的数据不可带有回车换行符)。音频数据要小于5MB。
DataLen	否	Integer	数据长度，非必填（此数据长度为数据未进行base64编码时的数据长度）。
HotwordId	否	String	热词id。用于调用对应的热词表，如果在调用语音识别服务时，不进行单独的热词id设置，自动生效默认热词；如果进行了单独的热词id设置，那么将生效单独设置的热词id。
FilterDirty	否	Integer	是否过滤脏词（目前支持中文普通话引擎）。0：不过滤脏词；1：过滤脏词；2：将脏词替换为 * 。默认值为 0。
FilterModal	否	Integer	是否过滤语气词（目前支持中文普通话引擎）。0：不过滤语气词；1：部分过滤；2：严格过滤 。默认值为 0。
ConvertNumMode	否	Integer	是否进行阿拉伯数字智能转换（目前支持中文普通话引擎）。0：不转换，直接输出中文数字，1：根据场景智能转换为阿拉伯数字，3: 打开数学相关数字转换。默认值为 1。
Extra	否	String	附加参数(该参数无意义，忽略即可)
FilterPunc	否	Integer	是否过滤标点符号（目前支持中文普通话引擎）。 0：不过滤，1：过滤句末标点，2：过滤所有标点。默认值为 0。
"""

'\nAction\t是\tString\t公共参数，本接口取值：CreateRecTask。\nVersion\t是\tString\t公共参数，本接口取值：2019-06-14。\nRegion\t否\tString\t公共参数，本接口不需要传递此参数。\nEngineModelType\t是\tString\t引擎模型类型。\n电话场景：\n• 8k_en：电话 8k 英语；\n• 8k_zh：电话 8k 中文普通话通用；\n非电话场景：\n• 16k_zh：16k 中文普通话通用；\n• 16k_zh_video：16k 音视频领域；\n• 16k_en：16k 英语；\n• 16k_ca：16k 粤语；\n• 16k_ja：16k 日语；\n• 16k_zh_edu 中文教育；\n• 16k_en_edu 英文教育；\n• 16k_zh_medical 医疗；\n• 16k_th 泰语；\nChannelNum\t是\tInteger\t识别声道数。1：单声道；2：双声道（仅支持 8k_zh 引擎模）。注意：录音识别会自动将音频转码为填写的识别声道数\nResTextFormat\t是\tInteger\t识别结果返回形式。0： 识别结果文本(含分段时间戳)； 1：词级别粒度的详细识别结果(不含标点，含语速值)；2：词级别粒度的详细识别结果（包含标点、语速值）\nSourceType\t是\tInteger\t语音数据来源。0：语音 URL；1：语音数据（post body）。\nSpeakerDiarization\t否\tInteger\t是否开启说话人分离，0：不开启，1：开启(仅支持8k_zh，16k_zh，16k_zh_video引擎模型，单声道音频)，默认值为 0。\nSpeakerNumber\t否\tInteger\t说话人分离人数（需配合开启说话人分离使用），取值范围：0-10，0代表自动分离（目前仅支持≤6个人），1-10代表指定说话人数分离。默认值为 0。\n注：话者分离目前是beta版本，请根据您的需要谨慎使用\nCallbackUrl\t否\tString\t回调 URL，用户自行搭建的用于接收识别结果的服务URL。如果用户使用轮询方式获取识别结果，则无需提交该参数。回调格式&内容详见：录音识别回调说明\nUrl\t否\tStri

In [8]:

try: 
    cred = credential.Credential(SecretId, SecretKey) 
    httpProfile = HttpProfile()
    httpProfile.endpoint = "asr.tencentcloudapi.com"

    clientProfile = ClientProfile()
    clientProfile.httpProfile = httpProfile
    client = asr_client.AsrClient(cred, "", clientProfile) 

    req = asr_models.DescribeTaskStatusRequest()
    params = {
        "TaskId": TaskId
    }
    req.from_json_string(json.dumps(params))

    resp = client.DescribeTaskStatus(req).to_json_string() 
    resp=json.loads(resp)
    print(resp)  

except TencentCloudSDKException as err: 
    print(err) 

{'Data': {'TaskId': 1529154133, 'Status': 1, 'StatusStr': 'doing', 'Result': '', 'ErrorMsg': '', 'ResultDetail': None}, 'RequestId': '031dd669-30ef-46aa-b308-c6a8aeee8849'}


In [9]:
resp["Data"]["Result"]

''

In [18]:
"""
https://console.cloud.tencent.com/api
/explorer?Product=asr&Version=2019-06-14&Action=SentenceRecognition&SignVersion=
一句话识别相关接口
"""
video_path="./video/output.mp3"
with open(video_path,"rb") as file:

    video=base64.b64encode(file.read()) 
Data=str(video , encoding='utf-8')
try: 
    cred = credential.Credential(SecretId, SecretKey) 
    httpProfile = HttpProfile()
    httpProfile.endpoint = "asr.tencentcloudapi.com"

    clientProfile = ClientProfile()
    clientProfile.httpProfile = httpProfile
    client = asr_client.AsrClient(cred, "", clientProfile) 

    req = asr_models.SentenceRecognitionRequest()
    params = {
        "ProjectId": 0,
        "SubServiceType": 2,
        "EngSerViceType": "16k_zh",
        "SourceType": 1,
        "VoiceFormat": "mp3",
        "UsrAudioKey": "uuid",
        "Data": Data,
        "FilterDirty": 0,
        "FilterModal": 0,
        "FilterPunc": 0,
        "ConvertNumMode": 0,
        "WordInfo": 0
    }
    req.from_json_string(json.dumps(params))

    resp = client.SentenceRecognition(req).to_json_string() 
    resp=json.loads(resp)
    print(resp)  
  
except TencentCloudSDKException as err: 
    print(err) 

{'Result': '基础语音课程。', 'AudioDuration': 1800, 'WordSize': 0, 'WordList': None, 'RequestId': 'df9e1597-9f6b-4d33-ba5d-6fe535c013a1'}
